In [35]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange

In [6]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [2]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2

# 데이터 가져오기

In [3]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]






In [4]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [6]:
# 2019년 - 2020년 / 디스커버리 / SH(운동화) 를 가지고 있는 매장 선택
start = time.time()

total_shop_stock_ori = '''
SELECT vfs.weekkey,vfs.shopkey ,vfs.partkey,vfs.scskey,vfs.brand,vfs.season,vfs.partcode,vfs.color,vfs.siz,vfs.week_takeout_qty,vfs.takeout_qty ,vfs.week_sale_qty ,vfs.sale_qty,vfs.week_sale_amt,vfs.stock_qty ,vfs.sale_amt,prdt.gender 
FROM ax.vw_f_scsshopweek vfs
INNER JOIN ax.vw_d_prdt prdt ON vfs.scskey  = prdt.scskey
WHERE vfs.brand = 'X' AND vfs.weekkey BETWEEN '2018-06-01' AND '2020-12-31' AND prdt.item ='SH'
'''


cur.execute(total_shop_stock_ori)
total_shop_stock_ori = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

total_shop_stock_ori = pd.DataFrame(total_shop_stock_ori)
total_shop_stock_ori.columns = ['weekkey','shopkey','partkey','scskey','brand','season','partcode','color','siz','week_takeout_qty','takeout_qty' ,'week_sale_qty' ,'sale_qty','week_sale_amt','stock_qty' ,'sale_amt','gender']
total_shop_stock = total_shop_stock_ori

time : 86.71158576011658


In [7]:
# 국내 / 오프라인 / 아울렛,상설 x / 디스커버리 / 2019년 - 2020년 전 기간 운영한 매장 

start = time.time()

shop_list = '''
SELECT shop_2019.shopkey 
FROM(SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%'  AND vds.brand = 'X' AND (closedate > '2019-12-31' OR closedate is NULL ) AND opendate < '2019-01-01'
LIMIT 1000) AS shop_2019
INNER JOIN (SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%' AND vds.brand = 'X' AND (closedate > '2020-12-31' OR closedate is NULL ) AND opendate < '2020-01-01'
) AS shop_2020
ON shop_2019.shopkey =shop_2020.shopkey
'''

cur.execute(shop_list)
shop_list = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
shop_list_pd = pd.DataFrame(shop_list)
shop_list_pd.columns = ['shopkey']

time : 10.406085014343262


In [8]:
# 국내 / 오프라인 / 아울렛,상설 x / 디스커버리 / 2019년 - 2020년 전 기간 운영한 매장  / 백화점

start = time.time()

shop_info = '''
SELECT shopkey ,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
FROM ax.vw_d_shop
WHERE type_ax_zone_nm ='국내' AND chnl_type_sfdc='오프라인'
;
'''

cur.execute(shop_info)
shop_info = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

shop_info_pd = pd.DataFrame(shop_info)
shop_info_pd.columns = ['shopkey' ,'shop_nm','shop_nm_sfdc','repr_shopcode','type_shop_nm','type_contract_nm','type_zone_nm','region_nm']

time : 10.82857060432434


# 전처리 관련 로직

## 1.상품 한정 ( 원하는 상품군 선택 , 모든 매장에 대한 주차별 값 생성)

In [4]:
# 빠져있는 주차 채워주는 로직 (원하는 상품 관련)

def zero_fill(shopkey, base, scskey_list,item):
    total=[]
    for col in base.columns:
        base[col].values[:] = 0
    for scskey in scskey_list:
        for shop in shopkey:
            frame = base.reset_index()
            target = item[(item['scskey']==scskey) & (item['shopkey']==shop )].groupby('weekkey').sum().reset_index()
            target = target.reset_index()

            for i in range(0,len(target)):
                for j in range(0,len(frame)):
                        if target.iloc[i].weekkey == frame.iloc[j].weekkey:
                            frame.iloc[j] = target.iloc[i]
                            break
            for i in range(1,len(frame)):
                if frame.iloc[i].takeout_qty ==0 and frame.iloc[i].sale_qty==0:
                    frame.iloc[i] = [ frame.iloc[i]['weekkey'], 0, frame.iloc[i-1]['takeout_qty'], 0, frame.iloc[i-1]['sale_qty'], frame.iloc[i-1]['stock_qty']]
            frame['shopkey'] =shop
            frame['ratio'] = 0
            for i in range(0,len(frame)):
                total_sale_qty= frame['week_sale_qty'].sum()
                a = frame.iloc[i]
                if a['sale_qty']==0 :
                    frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0]
                else:
                    if a['sale_qty'] / total_sale_qty > 1:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1]
                    else:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]
            else:
                frame['scskey']=scskey
            frame = pd.merge(frame, shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
            total.append(frame)
    return total


In [ ]:
# 2019  /버킷 디워커 230~ 270 사이즈 / BE 컬러 관련 매장 입출판재컬러 관련 매장 입출판재
scskey_list = ['X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BE240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,)

In [ ]:
shop_weeks_buket_df = pd.concat(shop_weeks_buket)

In [62]:
# 2020  /버킷 디워커 V2 230~ 270 사이즈 / BE 컬러 관련 매장 입출판재컬러 관련 매장 입출판재

scskey_list = ['X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA5011OW240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)

NameError: name 'zero_fill' is not defined

In [ ]:
shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

## 2.전체 매장 입출판재 

In [10]:
df_INNER_JOIN = pd.merge(shop_list_pd, total_shop_stock, left_on='shopkey', right_on='shopkey', how='inner')

In [11]:
shop_stock_2019 = df_INNER_JOIN[(df_INNER_JOIN['weekkey']< '20200101') & (df_INNER_JOIN['weekkey']> '20181231') ]
shop_stock_2020 = df_INNER_JOIN[df_INNER_JOIN['weekkey']> '20191231' ]

In [12]:
# 실제 매장 입고량과 판매량 구하는 로직(2019)
shop_stock_key_2019 = shop_stock_2019['shopkey'].unique()

total_sum_salty_2019 = []
total_sum_takeout_2019 = []
total_return_salty_2019=[]
total_sum_take_ware_2019=[]

for i in trange(0,len(shop_stock_key_2019)):
    takeout_qty = 0
    sale_qty = 0
    take_ware = 0
    return_salty=0
    
    shop_stock= shop_stock_2019[shop_stock_2019['shopkey']==shop_stock_key_2019[i]]
    shop_scs_key_2019 = shop_stock['scskey'].unique()
    for j in shop_scs_key_2019:
        takeout_qty+= shop_stock[shop_stock['scskey']==j]['takeout_qty'].max()
        sale_qty+= shop_stock[shop_stock['scskey']==j]['sale_qty'].max()
        take_ware += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_takeout_qty']<0)]['week_takeout_qty'].sum())
        return_salty += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_sale_qty']<0)]['week_sale_qty'].sum())
    total_sum_takeout_2019.append(takeout_qty)
    total_sum_salty_2019.append(sale_qty)
    total_sum_take_ware_2019.append(take_ware)
    total_return_salty_2019.append(return_salty)
df3 = pd.DataFrame([ x for x in zip(shop_stock_key_2019,total_sum_salty_2019,total_sum_takeout_2019,total_sum_take_ware_2019,total_return_salty_2019)])
df3.columns = ['shopkey','salty2019','takeout2019','take_ware2019','return_salty2019']
df3['salty2019'] = df3['salty2019'] + df3['return_salty2019']
print(df3)

  0%|                                                                                          | 0/143 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 실제 매장 입고량과 판매량 구하는 로직(2020)

shop_stock_key_2020 = shop_stock_2020['shopkey'].unique()

total_sum_salty2020 = []
total_sum_takeout2020 = []
total_return_salty2020=[]
total_sum_take_ware2020=[]

for i in trange(0,len(shop_stock_key_2020)):
    takeout_qty = 0
    sale_qty = 0
    take_ware = 0
    return_salty=0
    
    shop_stock= shop_stock_2020[shop_stock_2020['shopkey']==shop_stock_key_2020[i]]
    shop_scs_key_2020 = shop_stock['scskey'].unique()
    for j in shop_scs_key_2020:
        takeout_qty+= shop_stock[shop_stock['scskey']==j]['takeout_qty'].max()
        sale_qty+= shop_stock[shop_stock['scskey']==j]['sale_qty'].max()
        take_ware += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_takeout_qty']<0)]['week_takeout_qty'].sum())
        return_salty += int(shop_stock[(shop_stock['scskey']==j)&(shop_stock['week_sale_qty']<0)]['week_sale_qty'].sum())
    total_sum_takeout2020.append(takeout_qty)
    total_sum_salty2020.append(sale_qty)
    total_sum_take_ware2020.append(take_ware)
    total_return_salty2020.append(return_salty)
df4 = pd.DataFrame([ x for x in zip(shop_stock_key_2020,total_sum_salty2020,total_sum_takeout2020,total_sum_take_ware2020,total_return_salty2020)])
df4.columns = ['shopkey','salty2020','takeout2020','take_ware2020','return_salty2020']
df4['salty2020'] = df4['salty2020'] + df4['return_salty2020']
df4

## 3.버킷디워커 / 버킷디워커 2 - 평균 재고 소진 비율 구하기

In [71]:
# 전체 매장 평균 판매 비율 buket
bk_department_weeks_mean = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점'].groupby('weekkey').mean()
bk_agency_weeks_mean = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='대리점'].groupby('weekkey').mean()

In [72]:
# 전체 매장 평균 판매 비율 buket
bk_department_weeks_mean = bk_department_weeks_mean.reset_index()
bk_agency_weeks_mean = bk_agency_weeks_mean.reset_index()
bk_department_weeks_mean['weekkey'] = pd.to_datetime(bk_department_weeks_mean['weekkey'])
bk_agency_weeks_mean['weekkey'] = pd.to_datetime(bk_agency_weeks_mean['weekkey'])

In [73]:
bk_department_weeks_mean = bk_department_weeks_mean.set_index('weekkey')
bk_agency_weeks_mean = bk_agency_weeks_mean.set_index('weekkey')

In [74]:
# 평균 주차별 재고 소진율 차이 (버킷 디워커) 

bk_department_weeks_mean['rise_ratio'] = 0
for i in range(0,len(bk_department_weeks_mean)-1):
    a = bk_department_weeks_mean.iloc[i]
    if i ==0:
        bk_department_weeks_mean.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5] , a[5]]
    else:
        a = bk_department_weeks_mean.iloc[i]
        bk_department_weeks_mean.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5] , bk_department_weeks_mean['ratio'].iloc[i] - bk_department_weeks_mean['ratio'].iloc[i-1] ]


In [75]:
bk_department_weeks_mean_df = bk_department_weeks_mean.copy()

In [76]:
# 전체 매장 평균 판매 비율 buket_v2
v2_department_weeks_mean = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점'].groupby('weekkey').mean()
v2_agency_weeks_mean = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='대리점'].groupby('weekkey').mean()

In [77]:
# 전체 매장 평균 판매 비율 buket_v2
v2_department_weeks_mean = v2_department_weeks_mean.reset_index()
v2_agency_weeks_mean = v2_agency_weeks_mean.reset_index()
v2_department_weeks_mean['weekkey'] = pd.to_datetime(v2_department_weeks_mean['weekkey'])
v2_agency_weeks_mean['weekkey'] = pd.to_datetime(v2_agency_weeks_mean['weekkey'])

In [78]:
v2_department_weeks_mean = v2_department_weeks_mean.set_index('weekkey')
v2_agency_weeks_mean = v2_agency_weeks_mean.set_index('weekkey')

In [79]:
# 평균 주차별 재고 소진율 차이 (버킷 디워커 v2) 

v2_department_weeks_mean['rise_ratio'] = 0
for i in range(0,len(v2_department_weeks_mean)-1):
    a = v2_department_weeks_mean.iloc[i]
    if i ==0:
        v2_department_weeks_mean.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5] , a[5]]
    else:
        a = v2_department_weeks_mean.iloc[i]
        v2_department_weeks_mean.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5] , v2_department_weeks_mean['ratio'].iloc[i] - v2_department_weeks_mean['ratio'].iloc[i-1] ]

In [80]:
v2_department_weeks_mean_df = v2_department_weeks_mean.copy()

## 4.매장별 평균 소진비율 구하기

In [23]:
# 버킷디워커 (2019)  + 백화점 
de_shop_weeks_buket_df = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']
bk_total_shop_ratio = []
for shop_key in de_shop_weeks_buket_df['shopkey'].unique():
    shop_week = de_shop_weeks_buket_df[de_shop_weeks_buket_df['shopkey']== shop_key].groupby('weekkey').sum()
    shop_week = shop_week.reset_index()
    shop_week['weekkey'] = pd.to_datetime(shop_week['weekkey'])
    
    shop_week['ratio'] = 0
    shop_week['shopkey'] = 0
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
#             shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
            shop_week.iloc[i] = [a[0],a[1],a[2] ,a[3] ,a[4] ,a[5], 0,shop_key,0]

        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5], 1,shop_key,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5], a['sale_qty'] / total_sale_qty,shop_key,0]
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[8]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    bk_total_shop_ratio.append(shop_week)


In [24]:
bk_total_shop_ratio = pd.concat(bk_total_shop_ratio)

In [25]:
bk_total_shop_ratio

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,shopkey,rise_ratio
0,2018-12-31,4,4,0,0,4,0.000000,X10056,0.000000
1,2019-01-07,0,4,0,0,4,0.000000,X10056,0.000000
2,2019-01-14,3,7,1,1,6,0.012195,X10056,0.012195
3,2019-01-21,0,7,1,2,5,0.024390,X10056,0.012195
4,2019-01-28,-1,6,1,3,3,0.036585,X10056,0.012195
...,...,...,...,...,...,...,...,...,...
48,2019-12-02,-1,229,0,226,3,0.995595,X10038,0.000000
49,2019-12-09,1,230,1,227,3,1.000000,X10038,0.004405
50,2019-12-16,0,230,-1,226,4,0.995595,X10038,-0.004405
51,2019-12-23,0,230,0,226,4,0.995595,X10038,0.000000


In [26]:
bk_total_shop_ratio_info = pd.merge(bk_total_shop_ratio,shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
bk_total_shop_ratio_info

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,shopkey,rise_ratio,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2018-12-31,4,4,0,0,4,0.000000,X10056,0.000000,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-01-07,0,4,0,0,4,0.000000,X10056,0.000000,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-01-14,3,7,1,1,6,0.012195,X10056,0.012195,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-01-21,0,7,1,2,5,0.024390,X10056,0.012195,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-01-28,-1,6,1,3,3,0.036585,X10056,0.012195,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546,2019-12-02,-1,229,0,226,3,0.995595,X10038,0.000000,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3547,2019-12-09,1,230,1,227,3,1.000000,X10038,0.004405,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3548,2019-12-16,0,230,-1,226,4,0.995595,X10038,-0.004405,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3549,2019-12-23,0,230,0,226,4,0.995595,X10038,0.000000,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [28]:
# 버킷디워커 v2
de_shop_weeks_v2_df = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']

v2_total_shop_ratio = []
for shop_key in de_shop_weeks_v2_df['shopkey'].unique():
    shop_week = de_shop_weeks_v2_df[de_shop_weeks_v2_df['shopkey']== shop_key].groupby('weekkey').sum()
    shop_week = shop_week.reset_index()
    shop_week['weekkey'] = pd.to_datetime(shop_week['weekkey'])
    
    shop_week['ratio'] = 0
    shop_week['shopkey'] = 0
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
#             shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
            shop_week.iloc[i] = [a[0],a[1],a[2] ,a[3] ,a[4] ,a[5], 0,shop_key,0]

        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5], 1,shop_key,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5], a['sale_qty'] / total_sale_qty,shop_key,0]
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[8]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    v2_total_shop_ratio.append(shop_week)

In [29]:
v2_total_shop_ratio = pd.concat(v2_total_shop_ratio)

In [30]:
v2_total_shop_ratio

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,shopkey,rise_ratio
0,2019-11-18,6,6,0,0,6,0.000000,X10056,0.000000
1,2019-11-25,0,6,1,1,5,0.013333,X10056,0.013333
2,2019-12-02,5,11,3,4,7,0.053333,X10056,0.040000
3,2019-12-09,2,13,1,5,8,0.066667,X10056,0.013333
4,2019-12-16,2,15,0,5,10,0.066667,X10056,0.000000
...,...,...,...,...,...,...,...,...,...
54,2020-11-30,2,348,2,312,36,0.975000,X10038,0.006250
55,2020-12-07,1,349,4,316,33,0.987500,X10038,0.012500
56,2020-12-14,0,349,0,316,33,0.987500,X10038,0.000000
57,2020-12-21,0,349,2,318,31,0.993750,X10038,0.006250


In [31]:
v2_total_shop_ratio_info = pd.merge(v2_total_shop_ratio,shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
v2_total_shop_ratio_info

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,shopkey,rise_ratio,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,6,6,0,0,6,0.000000,X10056,0.000000,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-11-25,0,6,1,1,5,0.013333,X10056,0.013333,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-12-02,5,11,3,4,7,0.053333,X10056,0.040000,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-12-09,2,13,1,5,8,0.066667,X10056,0.013333,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-12-16,2,15,0,5,10,0.066667,X10056,0.000000,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,2020-11-30,2,348,2,312,36,0.975000,X10038,0.006250,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3949,2020-12-07,1,349,4,316,33,0.987500,X10038,0.012500,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3950,2020-12-14,0,349,0,316,33,0.987500,X10038,0.000000,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
3951,2020-12-21,0,349,2,318,31,0.993750,X10038,0.006250,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


# 전체 매장 입출판재 + 매장 info 데이터

In [32]:
shop_total_sales_info = pd.merge(shop_total_sales,shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
shop_total_sales_info

,shopkey,총 판매량(2019),총 입고량(2019),총 판매량(2020),총 입고량(2020),판매수량 비율(20/19),입고수량 비율(20/19),판매 비율(19/19),판매 비율(20/20),shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,X10070,2842,4298,8776,11012,3.087966,2.562122,0.661238,0.796949,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
1,X10057,2842,4881,7257,9722,2.553483,1.991805,0.582258,0.746451,현대목동,디스커버리 현대 목동점,XH0014,일반매장,백화점특정,백화점,서울
2,X10050,5668,8376,14182,17873,2.502117,2.133835,0.676695,0.793487,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
3,X10027,2984,4782,7036,11932,2.357909,2.495190,0.624007,0.589675,롯데평촌,디스커버리 롯데 평촌점,XH0222,일반매장,백화점특정,백화점,경기
4,X10006,4823,6728,11168,13899,2.315571,2.065844,0.716855,0.803511,AK분당,디스커버리 AK 분당점,XH0100,일반매장,백화점특정,백화점,경기
5,X50057,797,2074,1839,3064,2.307403,1.477338,0.384282,0.600196,여천(대-위),디스커버리 여천점,XH0061,일반매장,대리점,대리점,전남
6,X10064,3726,5409,8488,10827,2.278046,2.001664,0.688852,0.783966,현대천호,디스커버리 현대 천호점,XH0253,일반매장,백화점특정,백화점,서울
7,X50001,1824,3257,4049,5758,2.219846,1.767885,0.560025,0.703196,목포하당(대-위),디스커버리 목포하당점,XH0004,일반매장,대리점,대리점,전남
8,X10056,1611,2841,3433,4801,2.130975,1.689898,0.567054,0.715059,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
9,X50005,1035,2311,2181,3444,2.107246,1.490264,0.447858,0.633275,여수(대-위),디스커버리 여수점,XH0005,일반매장,대리점,대리점,전남


In [33]:
# 백화점 / 대리점 추출

department_total_sales = shop_total_sales_info[shop_total_sales_info['type_zone_nm']=='백화점']
agency_total_sales = shop_total_sales_info[shop_total_sales_info['type_zone_nm']=='대리점']

# 그룹화

## 1.분위수별

In [34]:
# 2019년 판매량 관련 분위수

d_Q1_2019 = department_total_sales['총 판매량(2019)'].quantile(.25)
d_Q3_2019 = department_total_sales['총 판매량(2019)'].quantile(.75)
d_IQR_2019 = d_Q3_2019 - d_Q1_2019
d_Q2_2019 = department_total_sales['총 판매량(2019)'].quantile(.5)
d_Q4_2019 = department_total_sales['총 판매량(2019)'].quantile(1)
d_Q0_2019 = department_total_sales[((department_total_sales['총 판매량(2019)']>d_Q3_2019+1.5*d_IQR_2019))]

In [35]:
d_q4_2019 = department_total_sales[department_total_sales['총 판매량(2019)']<d_Q1_2019]
d_q3_2019 = department_total_sales[(department_total_sales['총 판매량(2019)']>d_Q1_2019)&(department_total_sales['총 판매량(2019)']<d_Q2_2019)]
d_q2_2019 = department_total_sales[(department_total_sales['총 판매량(2019)']>d_Q2_2019)&(department_total_sales['총 판매량(2019)']<d_Q3_2019)]
d_q1_2019 = department_total_sales[(department_total_sales['총 판매량(2019)']>d_Q3_2019)&(department_total_sales['총 판매량(2019)']<d_Q3_2019+1.5*d_IQR_2019)]
d_q0_2019 = department_total_sales[((department_total_sales['총 판매량(2019)']>d_Q3_2019+1.5*d_IQR_2019))]

In [36]:
# 2020 년 판매량 관련 분위수

d_Q1_2020 = department_total_sales['총 판매량(2020)'].quantile(.25)
d_Q3_2020 = department_total_sales['총 판매량(2020)'].quantile(.75)
d_IQR_2020 = d_Q3_2020 - d_Q1_2020
d_Q2_2020 = department_total_sales['총 판매량(2020)'].quantile(.5)
d_Q4_2020 = department_total_sales['총 판매량(2020)'].quantile(1)
d_Q0_2020 = department_total_sales[((department_total_sales['총 판매량(2020)']>d_Q3_2020+1.5*d_IQR_2020))]

In [37]:
d_q4_2020 = department_total_sales[department_total_sales['총 판매량(2020)']<d_Q1_2020]
d_q3_2020 = department_total_sales[(department_total_sales['총 판매량(2020)']>d_Q1_2020)&(department_total_sales['총 판매량(2020)']<d_Q2_2020)]
d_q2_2020 = department_total_sales[(department_total_sales['총 판매량(2020)']>d_Q2_2020)&(department_total_sales['총 판매량(2020)']<d_Q3_2020)]
d_q1_2020 = department_total_sales[(department_total_sales['총 판매량(2020)']>d_Q3_2020)&(department_total_sales['총 판매량(2020)']<d_Q3_2020+1.5*d_IQR_2020)]
d_q0_2020 = department_total_sales[((department_total_sales['총 판매량(2020)']>d_Q3_2020+1.5*d_IQR_2020))]

In [38]:
d_q4_2019['quantile'] = 'Q4'
d_q3_2019['quantile'] = 'Q3'
d_q2_2019['quantile'] = 'Q2'
d_q1_2019['quantile'] = 'Q1'
d_q0_2019['quantile'] = 'Q0'

<ipython-input-38-10d89d940764>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_q4_2019['quantile'] = 'Q4'
<ipython-input-38-10d89d940764>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_q3_2019['quantile'] = 'Q3'
<ipython-input-38-10d89d940764>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [39]:
d_2019_quantile = [d_q0_2019,d_q1_2019,d_q2_2019,d_q3_2019,d_q4_2019]
d_2019_quantile = pd.concat(d_2019_quantile)

In [40]:
d_q4_2020['quantile'] = 'Q4'
d_q3_2020['quantile'] = 'Q3'
d_q2_2020['quantile'] = 'Q2'
d_q1_2020['quantile'] = 'Q1'
d_q0_2020['quantile'] = 'Q0'

<ipython-input-40-96223640cad9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_q4_2020['quantile'] = 'Q4'
<ipython-input-40-96223640cad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_q3_2020['quantile'] = 'Q3'
<ipython-input-40-96223640cad9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [41]:
d_2020_quantile = [d_q0_2020,d_q1_2020,d_q2_2020,d_q3_2020,d_q4_2020]
d_2020_quantile = pd.concat(d_2020_quantile)

In [42]:
# 각 분위수 설정

bk_total_shop_ratio_quantile= pd.merge(bk_total_shop_ratio, d_2019_quantile[['shopkey','quantile']], left_on='shopkey', right_on='shopkey', how='inner')
bk_quantile_shop = bk_total_shop_ratio_quantile.groupby(['weekkey','quantile']).sum().reset_index()

In [43]:
# 분위수 별 버킷 디워커 - 카운트 + ratio 생성


bk_total_quantile = []
count_list = []
rise_ratio_list = []

for quantile in bk_quantile_shop['quantile'].unique():
    count = 0
    shop_week = bk_quantile_shop[bk_quantile_shop['quantile']== quantile]
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty,0]
                
        if shop_week.iloc[i]['ratio'] > bk_department_weeks_mean_df.iloc[i]['ratio']:
            count += 1
        
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[7]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    else:
        count_list.append(count)
        bk_total_quantile.append(shop_week)


<ipython-input-43-1a8d71c540ef>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_week['rise_ratio'] = 0
C:\Users\AD0486\anaconda3\lib\site-packages\pandas\core\indexing.py:1760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [44]:
bk_total_quantile_df = pd.concat(bk_total_quantile)

In [45]:
# 분위수 카운트

bk_quantile_count = pd.DataFrame([ x for x in zip(bk_quantile_shop['quantile'],count_list)])
bk_quantile_count.columns=['quantile(2019)','count(2019)']

In [46]:
# 각 분위수 설정

v2_total_shop_ratio_quantile= pd.merge(v2_total_shop_ratio, d_2020_quantile[['shopkey','quantile']], left_on='shopkey', right_on='shopkey', how='inner')
v2_quantile_shop = v2_total_shop_ratio_quantile.groupby(['weekkey','quantile']).sum().reset_index()

In [47]:
# 분위수 별 버킷 디워커 v2 - 카운트 + ratio 생성

v2_total_quantile = []
count_list = []
for quantile in v2_quantile_shop['quantile'].unique():
    count = 0
    shop_week = v2_quantile_shop[v2_quantile_shop['quantile']== quantile]
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty,0]
                
        if shop_week.iloc[i]['ratio'] > v2_department_weeks_mean.iloc[i]['ratio']:
            count += 1
        
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[7]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    else:
        count_list.append(count)
        v2_total_quantile.append(shop_week) 

<ipython-input-47-504497e6d0d6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_week['rise_ratio'] = 0


In [48]:
v2_total_quantile_df = pd.concat(v2_total_quantile)

In [49]:
v2_quantile_count = pd.DataFrame([ x for x in zip(v2_quantile_shop['quantile'],count_list)])
v2_quantile_count.columns=['quantile(2020)','count(2020)']

## 2. 매장별

In [50]:
# 버킷 디워커 매장별 평균 소진비율 이상 개수 카운트 

count_list = []
for shop_key in bk_total_shop_ratio_info['shopkey'].unique():
    count = 0
    shop_week = bk_total_shop_ratio[bk_total_shop_ratio['shopkey']== shop_key]
    for i in range(0,len(shop_week)):
        if shop_week.iloc[i]['ratio'] > bk_department_weeks_mean_df.iloc[i]['ratio']:
            count += 1
    else:
        count_list.append(count)

In [51]:
shop_bk_count = pd.DataFrame([ x for x in zip(bk_total_shop_ratio_info['shopkey'].unique(),count_list)])
shop_bk_count.columns=['shopkey','ratio_count(2019)']

In [52]:
bk_total_shop_ratio_info_count = pd.merge(bk_total_shop_ratio_info,shop_bk_count, left_on='shopkey', right_on='shopkey', how='inner')

In [53]:
# 버킷 디워커 v2 매장별 평균 소진비율 이상 개수 카운트 

count_list = []
for shop_key in v2_total_shop_ratio_info['shopkey'].unique():
    count = 0
    shop_week = v2_total_shop_ratio_info[v2_total_shop_ratio_info['shopkey']== shop_key]
    for i in range(0,len(shop_week)):
        if shop_week.iloc[i]['ratio'] > v2_department_weeks_mean.iloc[i]['ratio']:
            count += 1
    else:
        count_list.append(count)

In [54]:
shop_v2_count = pd.DataFrame([ x for x in zip(v2_total_shop_ratio_info['shopkey'].unique(),count_list)])
shop_v2_count.columns=['shopkey','ratio_count(2020)']

In [55]:
v2_total_shop_ratio_info_count = pd.merge(v2_total_shop_ratio_info,shop_v2_count, left_on='shopkey', right_on='shopkey', how='inner')

## 3.지역별

In [56]:
bk_region_shop = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점'].groupby(['weekkey','region_nm']).sum().reset_index()

In [57]:
# 버킷 디워커 카운트

bk_total_region = []
count_list = []
for region in bk_region_shop['region_nm'].unique():
    count = 0
    shop_week = bk_region_shop[bk_region_shop['region_nm']== region]
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty,0]
                
        if shop_week.iloc[i]['ratio'] > bk_department_weeks_mean_df.iloc[i]['ratio']:
            count += 1
        
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[7]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    else:
        count_list.append(count)
        bk_total_region.append(shop_week) 

<ipython-input-57-3d55a36dc509>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_week['rise_ratio'] = 0


In [58]:
# 지역별 버킷디워커 재고 소진율

bk_total_region_df = pd.concat(bk_total_region)

In [59]:
bk_region_count = pd.DataFrame([ x for x in zip(bk_region_shop['region_nm'].unique(),count_list)])
bk_region_count.columns = ['지역','개수']

In [61]:
v2_region_shop = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점'].groupby(['weekkey','region_nm']).sum().reset_index()

In [62]:
# 버킷 디워커 v2 카운트

v2_total_region = []

count_list=[]
for region in v2_region_shop['region_nm'].unique():
    count = 0
    shop_week = v2_region_shop[v2_region_shop['region_nm']== region]
    shop_week['rise_ratio'] = 0
    
    for i in range(0,len(shop_week)):
        total_sale_qty= shop_week['week_sale_qty'].sum()
        a = shop_week.iloc[i]
        if a['sale_qty']==0 :
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0,0]
        else:
            if a['sale_qty'] / total_sale_qty > 1:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1,0]
            else:
                shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty,0]
                
        if shop_week.iloc[i]['ratio'] > v2_department_weeks_mean.iloc[i]['ratio']:
            count += 1
        
        a = shop_week.iloc[i]
        if i == 0:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],a[7]]
        else:
            shop_week.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] ,a[7],shop_week.iloc[i]['ratio'] - shop_week.iloc[i-1]['ratio']]

    else:
        count_list.append(count)
        v2_total_region.append(shop_week) 

<ipython-input-62-c7cb8b6bc340>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_week['rise_ratio'] = 0
C:\Users\AD0486\anaconda3\lib\site-packages\pandas\core\indexing.py:1760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [63]:
# 지역별 버킷디워커 v2 재고 소진율

v2_total_region_df = pd.concat(v2_total_region)

In [64]:
v2_region_count = pd.DataFrame([ x for x in zip(v2_region_shop['region_nm'].unique(),count_list)])
v2_region_count.columns = ['지역','개수']

## 파일 생성

In [82]:
bk_total_region_df = pd.concat(bk_total_region)
bk_total_quantile_df = pd.concat(bk_total_quantile)
bk_total_shop_ratio_info_count = pd.merge(bk_total_shop_ratio_info,shop_bk_count, left_on='shopkey', right_on='shopkey', how='inner')
bk_department_weeks_mean_df = bk_department_weeks_mean.copy()
v2_total_region_df = pd.concat(v2_total_region)
v2_total_shop_ratio_info_count = pd.merge(v2_total_shop_ratio_info,shop_v2_count, left_on='shopkey', right_on='shopkey', how='inner')
v2_total_quantile_df = pd.concat(v2_total_quantile)
v2_department_weeks_mean_df = v2_department_weeks_mean.copy()

In [83]:
bk_total_region_df.rise_ratio = round((bk_total_region_df.rise_ratio*100),2)
v2_total_region_df.rise_ratio = round((v2_total_region_df.rise_ratio*100),2)
v2_total_shop_ratio_info_count.rise_ratio = round((v2_total_shop_ratio_info_count.rise_ratio*100),2)
bk_total_shop_ratio_info_count.rise_ratio = round((bk_total_shop_ratio_info_count.rise_ratio*100),2)
v2_total_quantile_df.rise_ratio = round((v2_total_quantile_df.rise_ratio*100),2)
bk_total_quantile_df.rise_ratio = round((bk_total_quantile_df.rise_ratio*100),2)
bk_department_weeks_mean_df.rise_ratio = round((bk_department_weeks_mean_df.rise_ratio*100),2)
v2_department_weeks_mean_df.rise_ratio = round((v2_department_weeks_mean_df.rise_ratio*100),2)

In [542]:
# 지역 

bk_total_region_df.to_csv('bk_total_region_df.csv')
v2_total_region_df.to_csv('v2_total_region_df.csv')
bk_region_count.to_csv('bk_region_count.csv')
v2_region_count.to_csv('v2_region_count.csv')

In [554]:
# 매장별

v2_total_shop_ratio_info_count.to_csv('v2_total_shop_ratio_info_count.csv')
bk_total_shop_ratio_info_count.to_csv('bk_total_shop_ratio_info_count.csv')


In [430]:
# 분위수

v2_total_quantile_df.to_csv('v2_total_quantile_df.csv')
bk_total_quantile_df.to_csv('bk_total_quantile_df.csv')


In [502]:
bk_quantile_count.to_csv('bk_quantile_count.csv')
v2_quantile_count.to_csv('v2_quantile_count.csv')


In [433]:
# 평균

bk_department_weeks_mean_df.to_csv('bk_department_weeks_mean.csv')
v2_department_weeks_mean_df.to_csv('v2_department_weeks_mean.csv')

# 04.08 재고 소진율 / 루시 검색량과의 상관관계

In [92]:
# bk와 검색량
bk_search = pd.read_csv("bk_search.csv")
# bk_search = bk_search.iloc[ :,1:]

In [93]:
# bk와 검색량
v2_search = pd.read_csv("v2_search.csv")
# v2_search = v2_search.iloc[ :,1:]

In [119]:
v2_sales  = v2_department_weeks_mean_df.reset_index()
bk_sales = bk_department_weeks_mean_df.reset_index()

In [120]:
# bk 평균
bk_search_sales = pd.concat([bk_sales,bk_search],axis=1)
bk_search_sales.corr()

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,rise_ratio,search
week_takeout_qty,1.000000,-0.452459,0.848690,-0.527246,0.101626,-0.531499,0.826818,0.543160
takeout_qty,-0.452459,1.000000,-0.429838,0.976450,0.521980,0.970906,-0.372547,-0.050986
week_sale_qty,0.848690,-0.429838,1.000000,-0.518339,0.165546,-0.525819,0.990022,0.634960
sale_qty,-0.527246,0.976450,-0.518339,1.000000,0.325665,0.999487,-0.454026,-0.145522
stock_qty,0.101626,0.521980,0.165546,0.325665,1.000000,0.303402,0.162358,0.351883
ratio,-0.531499,0.970906,-0.525819,0.999487,0.303402,1.000000,-0.460568,-0.154628
rise_ratio,0.826818,-0.372547,0.990022,-0.454026,0.162358,-0.460568,1.000000,0.642394
search,0.543160,-0.050986,0.634960,-0.145522,0.351883,-0.154628,0.642394,1.000000


In [121]:
# v2 평균이랑 비슷 
v2_search_sales = pd.concat([v2_sales,v2_search],axis=1)
v2_search_sales.corr()

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,rise_ratio,search
week_takeout_qty,1.000000,-0.690171,0.762397,-0.695516,-0.546322,-0.701373,0.744127,0.259479
takeout_qty,-0.690171,1.000000,-0.481496,0.998746,0.877698,0.998121,-0.495304,0.107543
week_sale_qty,0.762397,-0.481496,1.000000,-0.482345,-0.408705,-0.489596,0.975519,0.487054
sale_qty,-0.695516,0.998746,-0.482345,1.000000,0.852604,0.999791,-0.498700,0.090087
stock_qty,-0.546322,0.877698,-0.408705,0.852604,1.000000,0.848080,-0.396282,0.260174
ratio,-0.701373,0.998121,-0.489596,0.999791,0.848080,1.000000,-0.506059,0.080121
rise_ratio,0.744127,-0.495304,0.975519,-0.498700,-0.396282,-0.506059,1.000000,0.462248
search,0.259479,0.107543,0.487054,0.090087,0.260174,0.080121,0.462248,1.000000


In [254]:
bk_search_quantile_list=[]
for i in bk_total_quantile_df['quantile'].unique():
#     print(len(v2_total_quantile_df[v2_total_quantile_df['quantile'] == i]))
    bk_search_quantile = pd.concat([ bk_total_quantile_df[bk_total_quantile_df['quantile'] == i].reset_index().iloc[:,1:],bk_search],axis=1)
    bk_search_quantile_list.append(bk_search_quantile.corr().iloc[-1][6])

In [255]:
bk_search_quantile_list

[0.39039069975259627,
 0.6001321616139836,
 0.6565907241931432,
 0.6425311929908867,
 0.4346860730694529]

In [273]:
bk_quantile_corr = pd.DataFrame(bk_search_quantile_list,bk_total_quantile_df['quantile'].unique())
bk_quantile_corr.columns=['corr']

In [275]:
bk_quantile_corr.to_csv('bk_quantile_corr.csv')

In [256]:
v2_search_quantile_list=[]
for i in v2_total_quantile_df['quantile'].unique():
#     print(len(v2_total_quantile_df[v2_total_quantile_df['quantile'] == i]))
    v2_search_quantile = pd.concat([ v2_total_quantile_df[v2_total_quantile_df['quantile'] == i].reset_index().iloc[:,1:],v2_search],axis=1)
    v2_search_quantile_list.append(v2_search_quantile.corr().iloc[-1][6])

In [276]:
v2_quantile_corr = pd.DataFrame(v2_search_quantile_list,v2_total_quantile_df['quantile'].unique())
v2_quantile_corr.columns=['corr']

In [280]:
v2_quantile_corr.to_csv('v2_quantile_corr.csv')

In [258]:
bk_search_region_list=[]
for i in bk_total_region_df['region_nm'].unique():
    bk_search_region = pd.concat([ bk_total_region_df[bk_total_region_df['region_nm'] == i].reset_index().iloc[:,1:] ,bk_search],axis=1)
    bk_search_region_list.append(bk_search_region.corr().iloc[-1][6])

In [277]:
bk_region_corr = pd.DataFrame(bk_search_region_list,bk_total_region_df['region_nm'].unique())
bk_region_corr.columns=['corr']

In [281]:
bk_region_corr.to_csv('bk_region_corr.csv')

In [264]:
v2_search_region_list=[]
for i in v2_total_region_df['region_nm'].unique():
    v2_search_region = pd.concat([ v2_total_region_df[v2_total_region_df['region_nm'] == i].reset_index().iloc[:,1:] ,v2_search],axis=1)
    v2_search_region_list.append(v2_search_region.corr().iloc[-1][6])

In [278]:
v2_region_corr = pd.DataFrame(v2_search_region_list,v2_total_region_df['region_nm'].unique())
v2_region_corr.columns=['corr']

In [282]:
v2_region_corr.to_csv('v2_region_corr.csv')

# 전채 물건 비율

In [9]:
total_shop_stock.groupby('partkey').sum()[['week_takeout_qty']].sort_values('week_takeout_qty',ascending = False)

,week_takeout_qty
partkey,
X20SDXSH70031,146793
X19FDXSH09961,125801
X19SDXSH09911,105242
X20SDXSHA5011,79812
X20SDXSHA3011,64476
X20SDXSHA1011,55564
X19SDXSH70931,52700
X18SDXSH70831,50256
X20SDXSHF4031,47676


In [14]:
# 국내 / 오프라인 / 아울렛,상설 x / 디스커버리 / 2019년 - 2020년 전 기간 운영한 매장 

start = time.time()

total_sale_prdt = '''
SELECT sales.shopkey ,sales.partkey,sales.scskey,sales.saledate,sales.cust_type_nm,sales.season,sales.partcode,sales.color,sales.siz,sales.cid,sales.qty,sales.amt_act,sales.amt_tag,prdt.sellprice,prdt.partcode_nm
FROM ax.mvw_f_sales sales
INNER JOIN ax.vw_d_prdt prdt ON sales.scskey  = prdt.scskey
WHERE sales.brand = 'X' AND sales.saledate BETWEEN To_date('2019-01-01', 'YYYY-MM-DD') AND
                           To_date('2020-12-31', 'YYYY-MM-DD') AND item = 'SH'
'''

cur.execute(total_sale_prdt)
total_sale_prdt = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
total_sale_prdt_pd = pd.DataFrame(total_sale_prdt)
total_sale_prdt_pd.columns = ['shopkey','partkey','scskey','saledate','cust_type_nm','season','partcode','color','siz','cid','qty','amt_act','amt_tag','sellprice','partcode_nm']

time : 46.56122159957886


In [17]:
total_sale_prdt_pd

,shopkey,partkey,scskey,saledate,cust_type_nm,season,partcode,color,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
0,X50037,X20SDXSH70031,X20SDXSH70031BK240,2020-05-24,소매,20S,DXSH70031,BK,240,X1727835,1,28000.00,39000,39000,[CO]샌드라인 3
1,X40001,X20SDXSH70031,X20SDXSH70031BK260,2020-08-25,소매,20S,DXSH70031,BK,260,None,4,156000.00,156000,39000,[CO]샌드라인 3
2,X10007,X19FDXSH09961,X19FDXSH09961BE255,2020-11-03,소매,19F,DXSH09961,BE,255,None,1,90300.00,129000,129000,[공통]버킷 디워커
3,X10032,X20SDXSH70031,X20SDXSH70031BK250,2020-01-19,소매,20S,DXSH70031,BK,250,X2671366,1,39000.00,39000,39000,[CO]샌드라인 3
4,X50065,X19FDXSH11941,X19FDXSH11941WH240,2019-10-26,소매,19F,DXSH11941,WH,240,X2414512,1,109000.00,109000,109000,[공통/아울렛]비글
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389899,X10070,X20SDXSH70031,X20SDXSH70031NY250,2020-06-13,소매,20S,DXSH70031,NY,250,None,1,39000.00,39000,39000,[CO]샌드라인 3
1389900,X80005,X19FDXSH09961,X19FDXSH09961BE260,2020-05-25,소매,19F,DXSH09961,BE,260,None,1,129000.00,129000,129000,[공통]버킷 디워커
1389901,X10060,X19SDXSH09911,X19SDXSH09911BE245,2019-05-12,소매,19S,DXSH09911,BE,245,None,1,129000.00,129000,129000,[공통]버킷 디워커
1389902,X50110,X19SDXSH70931,X19SDXSH70931WH250,2019-03-16,소매,19S,DXSH70931,WH,250,None,1,39000.00,39000,39000,[공통]샌드라인 2


In [22]:
total_sale_prdt_pd['saledate'] = pd.to_datetime(total_sale_prdt_pd['saledate'])


In [25]:
total_sale_2020_pd = total_sale_prdt_pd[total_sale_prdt_pd['saledate']>'2019.12.31']
total_sale_2020_pd

,shopkey,partkey,scskey,saledate,cust_type_nm,season,partcode,color,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
0,X50037,X20SDXSH70031,X20SDXSH70031BK240,2020-05-24,소매,20S,DXSH70031,BK,240,X1727835,1,28000.00,39000,39000,[CO]샌드라인 3
1,X40001,X20SDXSH70031,X20SDXSH70031BK260,2020-08-25,소매,20S,DXSH70031,BK,260,None,4,156000.00,156000,39000,[CO]샌드라인 3
2,X10007,X19FDXSH09961,X19FDXSH09961BE255,2020-11-03,소매,19F,DXSH09961,BE,255,None,1,90300.00,129000,129000,[공통]버킷 디워커
3,X10032,X20SDXSH70031,X20SDXSH70031BK250,2020-01-19,소매,20S,DXSH70031,BK,250,X2671366,1,39000.00,39000,39000,[CO]샌드라인 3
5,X50113,X20FDXSHH4041,X20FDXSHH4041WH230,2020-12-02,소매,20F,DXSHH4041,WH,230,None,2,130800.00,218000,109000,조거 클래식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389897,X50117,X20SDXSHA2011,X20SDXSHA2011BK275,2020-12-27,소매,20S,DXSHA2011,BK,275,None,1,139000.00,139000,139000,[CO]버킷 디워커 V2
1389898,X80005,X20SDXSHG2031,X20SDXSHG2031BK235,2020-10-14,소매,20S,DXSHG2031,BK,235,None,1,109000.00,109000,109000,[CO]브릭
1389899,X10070,X20SDXSH70031,X20SDXSH70031NY250,2020-06-13,소매,20S,DXSH70031,NY,250,None,1,39000.00,39000,39000,[CO]샌드라인 3
1389900,X80005,X19FDXSH09961,X19FDXSH09961BE260,2020-05-25,소매,19F,DXSH09961,BE,260,None,1,129000.00,129000,129000,[공통]버킷 디워커


In [26]:
total_sale_2019_pd = total_sale_prdt_pd[total_sale_prdt_pd['saledate']<'2020.01.01']
total_sale_2019_pd

,shopkey,partkey,scskey,saledate,cust_type_nm,season,partcode,color,siz,cid,qty,amt_act,amt_tag,sellprice,partcode_nm
4,X50065,X19FDXSH11941,X19FDXSH11941WH240,2019-10-26,소매,19F,DXSH11941,WH,240,X2414512,1,109000.00,109000,109000,[공통/아울렛]비글
6,X50130,X19FDXSH09961,X19FDXSH09961OW260,2019-12-25,소매,19F,DXSH09961,OW,260,None,1,129000.00,129000,129000,[공통]버킷 디워커
14,X30039,X19FDXSH70941,X19FDXSH70941NY250,2019-10-05,소매,19F,DXSH70941,NY,250,None,1,39000.00,39000,39000,[공통/아울렛]샌드라인 2
23,X50091,X19SDXSH09911,X19SDXSH09911GR230,2019-08-29,소매,19S,DXSH09911,GR,230,X2823789,1,119000.00,129000,129000,[공통]버킷 디워커
28,X50111,X19SDXSH09911,X19SDXSH09911GR265,2019-09-09,소매,19S,DXSH09911,GR,265,X2839383,1,119000.00,129000,129000,[공통]버킷 디워커
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389890,X10022,X19FDXSH78941,X19FDXSH78941WH240,2019-11-18,소매,19F,DXSH78941,WH,240,None,1,39000.00,39000,39000,[공통/아울렛]샌드 마크
1389891,X10022,X19FDXSH13961,X19FDXSH13961BE230,2019-10-11,소매,19F,DXSH13961,BE,230,None,1,109000.00,109000,109000,[공통/아울렛]조거 플렉스
1389893,X80005,X19FDXSH09961,X19FDXSH09961WH260,2019-08-30,소매,19F,DXSH09961,WH,260,None,1,129000.00,129000,129000,[공통]버킷 디워커
1389901,X10060,X19SDXSH09911,X19SDXSH09911BE245,2019-05-12,소매,19S,DXSH09911,BE,245,None,1,129000.00,129000,129000,[공통]버킷 디워커


In [34]:
total_shop_stock[total_shop_stock['weekkey']<'2020.01.01'].groupby('partkey').sum().week_takeout_qty.sort_values(ascending=False)

partkey
X20SDXSH70031    146793
X19FDXSH09961    125801
X19SDXSH09911    105242
X20SDXSHA5011     79812
X20SDXSHA3011     64476
X20SDXSHA1011     55564
X19SDXSH70931     52700
X18SDXSH70831     50256
X20SDXSHF4031     47676
X19FDXSH70941     41261
X18FDXSH70841     35159
X20SDXSHG1031     34615
X20SDXSHF6031     32579
X19FDXSH04961     32092
X19SDXSH82931     27624
X20SDXSHA2011     25108
X20SDXSH9C031     18105
X20SDXSHH1031     17925
X19SDXSH11911     17425
X20SDXSH9D031     16245
X20SDXSHE1031     15632
X19FDXSH11961     14590
X20SDXSHB2031     14503
X20SDXSHB1031     14029
X20SDXSHB4031     13157
X19FDXSH13961     13149
X19SDXSH04931     12829
X20SDXSH9B031     12253
X20SDXSH9A031     11243
X18FDXSH08841     11098
X18SDXSH80831     10828
X19SDXSH76931     10721
X19FDXSH11941     10193
X21SDXSHA1111      9745
X20SDXSHR2031      9639
X18FDXSH11841      9542
X19SDXSH13911      9184
X20SDXSHQ1031      8792
X20SDXSHF5031      8634
X18FDXSH16841      8493
X20SDXSHF2031      8447
X18SDXSH

In [39]:
bucket_dwalker_v2['weekkey'] = pd.to_datetime(bucket_dwalker_v2['weekkey'])


In [42]:
bucket_dwalker_v2_zero = bucket_dwalker_v2[ (bucket_dwalker_v2['stock_qty']==0) & (bucket_dwalker_v2['weekkey']<'2020-08-01') ]

In [56]:
bucket_dwalker_v2_zero.head()

,shopkey,weekkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
43,X10056,2020-03-16,X20SDXSHA3011,X20SDXSHA3011BE280,X,20S,DXSHA3011,BE,280,0,2,1,2,139000.0,0,278000.0,X
56,X10056,2019-12-16,X20SDXSHA1011,X20SDXSHA1011WH225,X,20S,DXSHA1011,WH,225,0,1,1,1,125100.0,0,125100.0,X
91,X10056,2019-12-16,X20SDXSHA5011,X20SDXSHA5011OW275,X,20S,DXSHA5011,OW,275,0,2,1,2,115000.0,0,254000.0,X
97,X10056,2019-11-18,X20SDXSHA1011,X20SDXSHA1011WH230,X,20S,DXSHA1011,WH,230,1,1,1,1,139000.0,0,139000.0,X
102,X10056,2019-11-18,X20SDXSHA1011,X20SDXSHA1011WH265,X,20S,DXSHA1011,WH,265,1,1,1,1,139000.0,0,139000.0,X


In [52]:
bucket_dwalker_v2_zero['shopkey'].unique()[0]

'X10056'

In [59]:
for i in bucket_dwalker_v2_zero['shopkey'].unique():
    bucket_dwalker_v2_zero[bucket_dwalker_v2_zero['shopkey']==i]
    break

In [61]:
bucket_dwalker_v2_zero[bucket_dwalker_v2_zero['shopkey']==i].partkey.unique()

array(['X20SDXSHA3011', 'X20SDXSHA1011', 'X20SDXSHA5011'], dtype=object)

In [95]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[(shop_weeks_buket_v2_df['type_zone_nm']=='백화점')]

In [92]:
shop_weeks_buket_v2_df_zero[shop_weeks_buket_v2_df_zero['weekkey']=='2020-12-09']

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm


In [94]:
shop_weeks_buket_v2_df_zero.sort_values('weekkey')

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
40887,2019-11-18,0,0,0,0,0,X10047,0.000000,X20SDXSHA5011OW265,마리오구로,디스커버리 마리오 구로점,XH0241,일반매장,백화점특정,백화점,서울
29500,2019-11-18,1,1,1,1,0,X10039,0.055556,X20SDXSHA5011OW260,AK원주,디스커버리 AK 원주점,XH0230,일반매장,백화점특정,백화점,강원
31388,2019-11-18,2,2,2,2,0,X10032,0.058824,X20SDXSHA5011OW260,롯데수원,디스커버리 롯데 수원점,XH0242,일반매장,백화점특정,백화점,경기
7965,2019-11-18,1,1,1,1,0,X10044,0.076923,X20SDXSHA5011OW230,롯데구리,디스커버리 롯데 구리점,XH0238,일반매장,백화점특정,백화점,경기
7139,2019-11-18,0,0,0,0,0,X10047,0.000000,X20SDXSHA5011OW230,마리오구로,디스커버리 마리오 구로점,XH0241,일반매장,백화점특정,백화점,서울
24013,2019-11-18,0,0,0,0,0,X10047,0.000000,X20SDXSHA5011OW240,마리오구로,디스커버리 마리오 구로점,XH0241,일반매장,백화점특정,백화점,서울
32450,2019-11-18,0,0,0,0,0,X10047,0.000000,X20SDXSHA5011OW260,마리오구로,디스커버리 마리오 구로점,XH0241,일반매장,백화점특정,백화점,서울
10030,2019-11-18,1,1,1,1,0,X10005,0.043478,X20SDXSHA5011OW235,롯데대전,디스커버리 롯데 대전점,XH0202,일반매장,백화점특정,백화점,대전
4602,2019-11-18,1,1,1,1,0,X10063,0.071429,X20SDXSHA5011OW230,롯데전주,디스커버리 롯데 전주점,XH0250,일반매장,백화점특정,백화점,전북
33512,2019-11-18,1,1,1,1,0,X10030,0.100000,X20SDXSHA5011OW260,롯데센텀,디스커버리 롯데 센텀점,XH0224,일반매장,백화점특정,백화점,부산


In [103]:
shop_weeks_buket_v2_df_dep[(shop_weeks_buket_v2_df_dep['stock_qty']==0 ) & (shop_weeks_buket_v2_df_dep['takeout_qty']>0 )].groupby('weekkey').sum()

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio
weekkey,,,,,,
2019-11-18,25,25,25,25,0,0.936240
2019-11-25,9,40,33,40,0,1.281482
2019-12-02,10,44,32,44,0,1.229025
2019-12-09,16,83,38,83,0,2.248580
2019-12-16,9,55,31,55,0,1.636948
2019-12-23,13,97,33,97,0,2.511151
2019-12-30,8,82,27,82,0,1.975032
2020-01-06,9,68,27,68,0,1.856562
2020-01-13,1,23,11,23,0,0.691481


In [110]:
shop_weeks_buket_v2_df_dep[(shop_weeks_buket_v2_df_dep['stock_qty']==0 ) & (shop_weeks_buket_v2_df_dep['weekkey']<'2020-05-18' ) & (shop_weeks_buket_v2_df_dep['takeout_qty']>0)].groupby('weekkey').count().sum()

week_takeout_qty    188
takeout_qty         188
week_sale_qty       188
sale_qty            188
stock_qty           188
shopkey             188
ratio               188
scskey              188
shop_nm             188
shop_nm_sfdc        188
repr_shopcode       188
type_shop_nm        188
type_contract_nm    188
type_zone_nm        188
region_nm           188
dtype: int64

In [120]:
shop_weeks_buket_v2_df[(shop_weeks_buket_v2_df['shopkey']=='X10056') & (shop_weeks_buket_v2_df['partkey']=='X20SDXSHA5011')].sort_values('weekkey')

KeyError: 'partkey'

In [143]:
shop_weeks_buket_v2_df[(shop_weeks_buket_v2_df['shopkey']=='X10070') & (shop_weeks_buket_v2_df['scskey']=='X20SDXSHA5011OW240')]

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
17641,2019-11-18,3,3,0,0,3,X10070,0.000000,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17642,2019-11-25,0,3,0,0,3,X10070,0.000000,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17643,2019-12-02,0,3,0,0,3,X10070,0.000000,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17644,2019-12-09,0,3,0,0,3,X10070,0.000000,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17645,2019-12-16,0,3,0,0,3,X10070,0.000000,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17646,2019-12-23,1,4,1,1,3,X10070,0.022222,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17647,2019-12-30,2,6,2,3,3,X10070,0.066667,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17648,2020-01-06,3,9,2,5,4,X10070,0.111111,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17649,2020-01-13,0,9,2,7,2,X10070,0.155556,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울
17650,2020-01-20,3,12,0,7,5,X10070,0.155556,X20SDXSHA5011OW240,신세계강남,디스커버리 신세계 강남점,XH0260,일반매장,백화점특정,백화점,서울


In [11]:
test1 = shop_weeks_buket_v2_df[(shop_weeks_buket_v2_df['shop_nm']=='롯데본점' ) & (shop_weeks_buket_v2_df['scskey']=='X20SDXSHA5011OW240')]

In [22]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [72]:
def make_sma_arr(window_num):
    ma_arr = np.array([])
#     for i in df_month.store_id.unique():
    df_set = test1
    ma_arr = np.concatenate((ma_arr, df_set.week_sale_qty.rolling(window=window_num).mean().values ))
        
    return ma_arr

In [73]:
sma_month = test1.copy()

sma_month.insert(7, 'amount_2ma', make_sma_arr(2))
sma_month.insert(8, 'amount_3ma', make_sma_arr(3))
sma_month.insert(9, 'amount_6ma', make_sma_arr(6))

In [74]:
print("2 winodw SMA MAE Score : ", mae(sma_month.amount_2ma.fillna(0), sma_month.week_sale_qty.values))
print("3 window SMA MAE Score : ", mae(sma_month.amount_3ma.fillna(0), sma_month.week_sale_qty.values))
print("6 window SMA MAE Score : ", mae(sma_month.amount_6ma.fillna(0), sma_month.week_sale_qty.values))

2 winodw SMA MAE Score :  0.711864406779661
3 window SMA MAE Score :  0.8418079096045198
6 window SMA MAE Score :  1.115819209039548


In [78]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = test1
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values))
    
    return arr_ewm

In [79]:
ewm_month = test1.copy()
ewm_month.insert(6, 'amount_2ewm', make_ewm_arr(test1, 2))
ewm_month.insert(7, 'amount_3ewm', make_ewm_arr(test1, 3))
ewm_month.insert(8, 'amount_6ewm', make_ewm_arr(test1, 6))

In [80]:
print("2 N EWM MAE Score : ", mae(ewm_month.amount_2ewm, sma_month.week_sale_qty.values))
print("3 N EWM MAE Score : ", mae(ewm_month.amount_3ewm, sma_month.week_sale_qty.values))
print("6 N EWM MAE Score : ", mae(ewm_month.amount_6ewm, sma_month.week_sale_qty.values))

2 N EWM MAE Score :  0.42481363328486504
3 N EWM MAE Score :  0.6195452265520786
6 N EWM MAE Score :  0.8689439569217453


In [32]:
print("6 N EWM MAE Score : ", mae(ewm_month.week_takeout_qty, sma_month.week_sale_qty.values))

6 N EWM MAE Score :  1.0677966101694916


In [33]:
ewm_month

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,amount_2ewm,amount_3ewm,amount_6ewm,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
19470,2019-11-18,2,2,1,1,1,1.000000,1.000000,1.000000,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19471,2019-11-25,2,4,0,1,3,0.250000,0.333333,0.416667,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19472,2019-12-02,1,5,3,4,1,2.153846,1.857143,1.577982,X10050,0.052632,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19473,2019-12-09,8,13,2,6,7,2.050000,1.933333,1.740991,X10050,0.078947,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19474,2019-12-16,1,14,1,7,7,1.347107,1.451613,1.480924,X10050,0.092105,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19475,2019-12-23,5,19,6,13,6,4.453297,3.761905,2.969831,X10050,0.171053,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19476,2019-12-30,2,21,1,14,7,2.150046,2.370079,2.348036,X10050,0.184211,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19477,2020-01-06,2,23,4,18,5,3.383537,3.188235,2.854332,X10050,0.236842,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19478,2020-01-13,4,27,2,20,7,2.461132,2.592955,2.597822,X10050,0.263158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19479,2020-01-20,0,27,1,21,6,1.487028,1.795699,2.124954,X10050,0.276316,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울


In [34]:
ewm_month.amount_2ewm

19470    1.000000
19471    0.250000
19472    2.153846
19473    2.050000
19474    1.347107
19475    4.453297
19476    2.150046
19477    3.383537
19478    2.461132
19479    1.487028
19480   -0.171000
19481    1.943004
19482    2.647668
19483    4.215890
19484    5.405297
19485    5.135099
19486    5.045033
19487    3.015011
19488    1.005004
19489    1.001668
19490   -0.332777
19491    1.222408
19492    0.407469
19493    0.135823
19494    0.711941
19495    2.237314
19496    1.412438
19497    0.470813
19498    0.156938
19499    1.385646
19500    0.461882
19501    1.487294
19502    0.495765
19503    0.831922
19504    0.277307
19505    0.092436
19506    2.030812
19507    0.676937
19508    0.225646
19509    0.075215
19510    0.025072
19511    0.675024
19512    0.891675
19513   -0.369442
19514   -0.123147
19515    0.625618
19516    0.208539
19517    0.069513
19518    0.689838
19519    0.896613
19520    0.965538
19521    0.321846
19522    2.107282
19523    0.702427
19524    0.234142
19525    0

In [76]:
ceil_list = [] 
floor_list = [] 
round_list = []
count = 0
for i in sma_month.amount_2ma:
    if count == 0:
        ceil_list.append(0)
        floor_list.append(0)
        round_list.append(0)

        count+=1
    else :
        ceil_list.append( math.ceil(i))
        floor_list.append( math.floor(i))
        round_list.append( round(i , 0) )
#     break

In [77]:
print("2 N EWM MAE Score : ", mae(ceil_list, sma_month.week_sale_qty.values))
print("3 N EWM MAE Score : ", mae(floor_list, sma_month.week_sale_qty.values))
print("6 N EWM MAE Score : ", mae(round_list, sma_month.week_sale_qty.values))

2 N EWM MAE Score :  0.6949152542372882
3 N EWM MAE Score :  0.7288135593220338
6 N EWM MAE Score :  0.7457627118644068


In [81]:
ceil_list = [] 
floor_list = [] 
round_list = []
count = 0
for i in ewm_month.amount_2ewm:
    if count == 0:
        ceil_list.append(0)
        floor_list.append(0)
        round_list.append(0)

        count+=1
    else :
        ceil_list.append( math.ceil(i))
        floor_list.append( math.floor(i))
        round_list.append( round(i , 0) )
#     break

In [86]:
print("2 N EWM MAE Score : ", mae(ceil_list, ewm_month.week_sale_qty.values))
print("3 N EWM MAE Score : ", mae(floor_list, ewm_month.week_sale_qty.values))
print("6 N EWM MAE Score : ", mae(round_list, ewm_month.week_sale_qty.values))

2 N EWM MAE Score :  0.6440677966101694
3 N EWM MAE Score :  0.5423728813559322
6 N EWM MAE Score :  0.3728813559322034


In [83]:
ewm_month

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,amount_2ewm,amount_3ewm,amount_6ewm,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
19470,2019-11-18,2,2,1,1,1,1.000000,1.000000,1.000000,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19471,2019-11-25,2,4,0,1,3,0.250000,0.333333,0.416667,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19472,2019-12-02,1,5,3,4,1,2.153846,1.857143,1.577982,X10050,0.052632,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19473,2019-12-09,8,13,2,6,7,2.050000,1.933333,1.740991,X10050,0.078947,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19474,2019-12-16,1,14,1,7,7,1.347107,1.451613,1.480924,X10050,0.092105,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19475,2019-12-23,5,19,6,13,6,4.453297,3.761905,2.969831,X10050,0.171053,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19476,2019-12-30,2,21,1,14,7,2.150046,2.370079,2.348036,X10050,0.184211,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19477,2020-01-06,2,23,4,18,5,3.383537,3.188235,2.854332,X10050,0.236842,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19478,2020-01-13,4,27,2,20,7,2.461132,2.592955,2.597822,X10050,0.263158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19479,2020-01-20,0,27,1,21,6,1.487028,1.795699,2.124954,X10050,0.276316,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울


In [87]:
sma_month

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,amount_2ma,amount_3ma,amount_6ma,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
19470,2019-11-18,2,2,1,1,1,X10050,NaN,NaN,NaN,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19471,2019-11-25,2,4,0,1,3,X10050,0.5,NaN,NaN,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19472,2019-12-02,1,5,3,4,1,X10050,1.5,1.333333,NaN,0.052632,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19473,2019-12-09,8,13,2,6,7,X10050,2.5,1.666667,NaN,0.078947,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19474,2019-12-16,1,14,1,7,7,X10050,1.5,2.000000,NaN,0.092105,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19475,2019-12-23,5,19,6,13,6,X10050,3.5,3.000000,2.166667,0.171053,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19476,2019-12-30,2,21,1,14,7,X10050,3.5,2.666667,2.166667,0.184211,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19477,2020-01-06,2,23,4,18,5,X10050,2.5,3.666667,2.833333,0.236842,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19478,2020-01-13,4,27,2,20,7,X10050,3.0,2.333333,2.666667,0.263158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19479,2020-01-20,0,27,1,21,6,X10050,1.5,2.333333,2.500000,0.276316,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울


In [88]:
test1

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
19470,2019-11-18,2,2,1,1,1,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19471,2019-11-25,2,4,0,1,3,X10050,0.013158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19472,2019-12-02,1,5,3,4,1,X10050,0.052632,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19473,2019-12-09,8,13,2,6,7,X10050,0.078947,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19474,2019-12-16,1,14,1,7,7,X10050,0.092105,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19475,2019-12-23,5,19,6,13,6,X10050,0.171053,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19476,2019-12-30,2,21,1,14,7,X10050,0.184211,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19477,2020-01-06,2,23,4,18,5,X10050,0.236842,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19478,2020-01-13,4,27,2,20,7,X10050,0.263158,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울
19479,2020-01-20,0,27,1,21,6,X10050,0.276316,X20SDXSHA5011OW240,롯데본점,디스커버리 롯데 본점,XH0248,일반매장,백화점특정,백화점,서울


In [107]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = shop_pd_scskey
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values))
    
    return arr_ewm

In [108]:
def make_sma_arr(window_num):
    ma_arr = np.array([])
#     for i in df_month.store_id.unique():
    df_set = shop_pd_scskey
    ma_arr = np.concatenate((ma_arr, df_set.week_sale_qty.rolling(window=window_num).mean().values ))
        
    return ma_arr

In [109]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']

In [110]:
scskey_unique = shop_weeks_buket_v2_df_dep['scskey'].unique()

In [111]:
shopkey_unique = shop_weeks_buket_v2_df_dep['shopkey'].unique()

In [ ]:
ceil_list = [] 
floor_list = [] 
round_list = []
count = 0
for i in ewm_month.amount_2ewm:
    if count == 0:
        ceil_list.append(0)
        floor_list.append(0)
        round_list.append(0)

        count+=1
    else :
        ceil_list.append( math.ceil(i))
        floor_list.append( math.floor(i))
        round_list.append( round(i , 0) )

In [115]:
# ceil
amount_2ma_list = []
amount_3ma_list = []
amount_6ma_list = []
amount_2ewm_list = []
amount_3ewm_list = []
amount_6ewm_list = []
takeout_qty_list = []
for i in shopkey_unique:
    shop_pd = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in scskey_unique:
        shop_pd_scskey = shop_pd[shop_pd['scskey']==j]
        sma_month = shop_pd_scskey.copy()
        sma_month.insert(7, 'amount_2ma', make_sma_arr(2))
        sma_month.insert(8, 'amount_3ma', make_sma_arr(3))
        sma_month.insert(9, 'amount_6ma', make_sma_arr(6))
        ma_list = ['amount_2ma','amount_3ma','amount_6ma']
        for i in range(0,3):
        ceil_list = [] 
        floor_list = [] 
        round_list = []
        count = 0
        for i in ewm_month.amount_2ewm:
            if count == 0:
                ceil_list.append(0)
                floor_list.append(0)
                round_list.append(0)

                count+=1
            else :
                ceil_list.append( math.ceil(i))
                floor_list.append( math.floor(i))
                round_list.append( round(i , 0) )
        

        amount_2ma_list.append(mae(sma_month.amount_2ma.fillna(0), sma_month.week_sale_qty.values))
        amount_3ma_list.append(mae(sma_month.amount_3ma.fillna(0), sma_month.week_sale_qty.values))
        amount_6ma_list.append(mae(sma_month.amount_6ma.fillna(0), sma_month.week_sale_qty.values))
        ewm_month = shop_pd_scskey.copy()
        ewm_month.insert(6, 'amount_2ewm', make_ewm_arr(ewm_month, 2))
        ewm_month.insert(7, 'amount_3ewm', make_ewm_arr(ewm_month, 3))
        ewm_month.insert(8, 'amount_6ewm', make_ewm_arr(ewm_month, 6))
        amount_2ewm_list.append(mae(ewm_month.amount_2ewm, sma_month.week_sale_qty.values))
        amount_3ewm_list.append(mae(ewm_month.amount_3ewm, ewm_month.week_sale_qty.values))
        amount_6ewm_list.append(mae(ewm_month.amount_6ewm, ewm_month.week_sale_qty.values))
        
        takeout_qty_list.append(mae(ewm_month.week_takeout_qty, ewm_month.week_sale_qty.values))


In [126]:
ceil_list = [] 
floor_list = [] 
round_list = []
count = 0
for i in amount_2ma_list:
    if count == 0:
        ceil_list.append(0)
        floor_list.append(0)
        round_list.append(0)

        count+=1
    else :
        ceil_list.append( math.ceil(i))
        floor_list.append( math.floor(i))
        round_list.append( round(i , 0) )
#     break
print("2 N EWM MAE Score : ", mae(ceil_list, ewm_month.week_sale_qty.values))
print("3 N EWM MAE Score : ", mae(floor_list, ewm_month.week_sale_qty.values))
print("6 N EWM MAE Score : ", mae(round_list, ewm_month.week_sale_qty.values))




# for i in range(0,len(amount_2ma_list)):
#     print( math.ceil(amount_2ma_list[i]) )

ValueError: operands could not be broadcast together with shapes (59,) (402,) 

In [118]:
print(sum(amount_2ma_list) / len(amount_2ma_list))
print(sum(amount_3ma_list) / len(amount_3ma_list))
print(sum(amount_6ma_list) / len(amount_6ma_list))
print(sum(amount_2ewm_list) / len(amount_2ewm_list))
print(sum(amount_3ewm_list) / len(amount_3ewm_list))
print(sum(amount_6ewm_list) / len(amount_6ewm_list))
print(sum(takeout_qty_list) / len(takeout_qty_list))

0.37045703684965026
0.4592995474604379
0.5540306939876884
0.22663271395917708
0.32776863455358285
0.4535644165185542
0.5652247238384354


In [ ]:
# ceil
amount_2ma_list = []
amount_3ma_list = []
amount_6ma_list = []
amount_2ewm_list = []
amount_3ewm_list = []
amount_6ewm_list = []
takeout_qty_list = []
for i in shopkey_unique:
    shop_pd = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in scskey_unique:
        shop_pd_scskey = shop_pd[shop_pd['scskey']==j]
        sma_month = shop_pd_scskey.copy()
        sma_month.insert(7, 'amount_2ma', make_sma_arr(2))
        sma_month.insert(8, 'amount_3ma', make_sma_arr(3))
        sma_month.insert(9, 'amount_6ma', make_sma_arr(6))
        amount_2ma_list.append(mae(sma_month.amount_2ma.fillna(0), sma_month.week_sale_qty.values))
        amount_3ma_list.append(mae(sma_month.amount_3ma.fillna(0), sma_month.week_sale_qty.values))
        amount_6ma_list.append(mae(sma_month.amount_6ma.fillna(0), sma_month.week_sale_qty.values))
        ewm_month = shop_pd_scskey.copy()
        ewm_month.insert(6, 'amount_2ewm', make_ewm_arr(ewm_month, 2))
        ewm_month.insert(7, 'amount_3ewm', make_ewm_arr(ewm_month, 3))
        ewm_month.insert(8, 'amount_6ewm', make_ewm_arr(ewm_month, 6))
        amount_2ewm_list.append(mae(ewm_month.amount_2ewm, sma_month.week_sale_qty.values))
        amount_3ewm_list.append(mae(ewm_month.amount_3ewm, ewm_month.week_sale_qty.values))
        amount_6ewm_list.append(mae(ewm_month.amount_6ewm, ewm_month.week_sale_qty.values))
        
        takeout_qty_list.append(mae(ewm_month.week_takeout_qty, ewm_month.week_sale_qty.values))


In [119]:
def make_wma_sub(data_frame, span_num):
    concat_3wek = pd.DataFrame(index=pd.to_datetime(['2021-01-05', '2021-01-12', '2021-01-19']))
    wma_sub = np.array([])

    for i in df_month.store_id.unique():
        df_set = pd.DataFrame(data_frame[data_frame.store_id == i].amount)
        # 비어있는 셀 만들어주기
        wma_train = pd.concat([df_set, concat_3mon], axis=0)
        
        # 세달 예측 후 합하기
        num_sub = np.array([wma_train.amount.ewm(span=span_num).mean()['2021-01':].sum()])
        wma_sub = np.concatenate((wma_sub, num_sub))
        
    df_wma_sub = pd.DataFrame({'store_id' : df_sub.store_id, 'amount' : wma_sub})
    return df_wma_sub

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,amount_2ewm,amount_3ewm,amount_6ewm,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
50445,2019-11-18,3,3,0,0,3,0.000000,0.000000,0.000000,X10038,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50446,2019-11-25,0,3,0,0,3,0.000000,0.000000,0.000000,X10038,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50447,2019-12-02,0,3,0,0,3,0.000000,0.000000,0.000000,X10038,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50448,2019-12-09,0,3,1,1,2,0.675000,0.533333,0.386261,X10038,0.022727,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50449,2019-12-16,1,4,0,1,3,0.223140,0.258065,0.250694,X10038,0.022727,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50450,2019-12-23,0,4,3,4,0,2.076923,1.650794,1.156512,X10038,0.090909,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50451,2019-12-30,3,7,1,5,2,1.358646,1.322835,1.107108,X10038,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50452,2020-01-06,1,8,0,5,3,0.452744,0.658824,0.767800,X10038,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50453,2020-01-13,0,8,0,5,3,0.150899,0.328767,0.537271,X10038,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50454,2020-01-20,0,8,1,6,2,0.716976,0.664712,0.674213,X10038,0.136364,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [127]:
sma_month

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,amount_2ma,amount_3ma,amount_6ma,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
50445,2019-11-18,3,3,0,0,3,X10038,NaN,NaN,NaN,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50446,2019-11-25,0,3,0,0,3,X10038,0.0,NaN,NaN,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50447,2019-12-02,0,3,0,0,3,X10038,0.0,0.000000,NaN,0.000000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50448,2019-12-09,0,3,1,1,2,X10038,0.5,0.333333,NaN,0.022727,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50449,2019-12-16,1,4,0,1,3,X10038,0.5,0.333333,NaN,0.022727,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50450,2019-12-23,0,4,3,4,0,X10038,1.5,1.333333,0.666667,0.090909,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50451,2019-12-30,3,7,1,5,2,X10038,2.0,1.333333,0.833333,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50452,2020-01-06,1,8,0,5,3,X10038,0.5,1.333333,0.833333,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50453,2020-01-13,0,8,0,5,3,X10038,0.0,0.333333,0.833333,0.113636,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50454,2020-01-20,0,8,1,6,2,X10038,0.5,0.333333,0.833333,0.136364,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
